<a href="https://colab.research.google.com/github/nhatduong01/SQL_on_BigQuery/blob/main/Basis/World_Bank_International_Education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working with `international_education` table on the `world_bank_intl_education` dataset

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
# Set up the Project Environment
import os
os.environ["GCLOUD_PROJECT"] = "BigQuerySQLExercises"

In [6]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

data_ref = client.dataset(dataset_id= 'world_bank_intl_education',
                          project = "bigquery-public-data")
dataset = client.get_dataset(data_ref)
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

country_series_definitions
country_summary
international_education
series_summary


In [10]:
# Fetch the table
table_ref = data_ref.table('international_education')

table = client.get_table(table_ref)
# Print table schema
table.schema

[SchemaField('country_name', 'STRING', 'NULLABLE', '', ()), SchemaField('country_code', 'STRING', 'NULLABLE', '', ()), SchemaField('indicator_name', 'STRING', 'NULLABLE', '', ()), SchemaField('indicator_code', 'STRING', 'NULLABLE', '', ()), SchemaField('value', 'FLOAT', 'NULLABLE', '', ()), SchemaField('year', 'INTEGER', 'NULLABLE', '', ())]


In [11]:
#List the first 10 rows
client.list_rows(table, max_results= 10).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Djibouti,DJI,"Enrolment in secondary vocational, female (num...",SE.SEC.ENRL.VO.FE,2217.000000,2016
1,Djibouti,DJI,"Gross graduation ratio from primary education,...",SE.PRM.CMPL.ZS,46.081841,2016
2,Dominica,DMA,Population of the official age for the last gr...,SP.PRM.GRAD.MA,470.000000,2016
3,Pakistan,PAK,Internet users (per 100 people),IT.NET.USER.P2,15.514558,2016
4,Liberia,LBR,SABER: (Tertiary Education) Policy Goal 4: Fin...,SABER.TER.GOAL4,1.000000,2017
5,Libya,LBY,Population growth (annual %),SP.POP.GROW,0.930675,2016
6,Suriname,SUR,Population of the official age for pre-primary...,SP.PRE.TOTL.FE.IN,9249.000000,2016
7,Suriname,SUR,SABER: (Education Management Information Syste...,SABER.EMIS.GOAL1.LVL2,2.000000,2016
8,Tajikistan,TJK,"Enrolment in primary education, female (number)",SE.PRM.ENRL.FE,346823.000000,2016
9,Tajikistan,TJK,GDP per capita (constant 2005 US$),NY.GDP.PCAP.KD,968.385603,2016


### 1) Goverment Expenditures on Education
Which countries spend the largest fraction of GDP on education?
`SE.XPD.TOTL.GD.ZS` indicator in the table specifies the amount of GDP goverment spent on education. We need a query to return the `AVG()` of GDP spent of each country from 2010 -2017.

In [17]:
query = """
        SELECT country_name,AVG(value) AS Amount
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE year <= 2017 AND YEAR >= 2010 AND indicator_code = 'SE.XPD.TOTL.GD.ZS'
        GROUP BY country_name
        ORDER BY Amount DESC
        """
query_job = client.query(query= query, project= 'bigquerysqlexercises')

GDP_expen = query_job.to_dataframe()
print(GDP_expen)

                 country_name     Amount
0                        Cuba  12.837270
1       Micronesia, Fed. Sts.  12.467750
2             Solomon Islands  10.001080
3                     Moldova   8.372153
4                     Namibia   8.349610
..                        ...        ...
152                  Cambodia   1.706404
153        West Bank and Gaza   1.503760
154               South Sudan   1.409726
155                    Monaco   1.409606
156  Central African Republic   1.214010

[157 rows x 2 columns]


In [23]:
# Lets see where VietNam is
GDP_expen[GDP_expen.country_name == 'Vietnam']

,country_name,Amount
49,Vietnam,5.282447
